In [ ]:
# run this once to set up

from IPython.display import clear_output
%pip install twarc
%pip install --upgrade tweepy==3.8.0
%pip install argparse
%pip install xtract
%pip install wget
%pip install ipywidgets
%pip install --upgrade emoji==1.7
%pip install --upgrade emot==2.1
clear_output()

In [ ]:
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd
import json
import tweepy
import datetime
import random
from twarc import Twarc2, expansions
from tweepy import OAuthHandler
from os.path import exists

In [ ]:
### Use your Twitter Developer keys and tokens ###
### Must use Twitter API v1.1 (Elevated Access of Twitter Developer Account) ### 

CONSUMER_KEY = "" 
CONSUMER_SECRET_KEY = ""
ACCESS_TOKEN_KEY = ""
ACCESS_TOKEN_SECRET_KEY = ""

#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)

# The lines below are just to test if the twitter credentials are correct
# Authenticate
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True)

if (not api):
    print ("Can't Authenticate")
#  sys.exit(-1)
else:
    print('Authentication OK')

# datasets that can be retrieved are from 07/26/2020 to 11/19/2022
dataset_date = datetime.date(2020,7,26)
dataset_end_date = datetime.date(2022,11,19)

In [ ]:
while dataset_date < dataset_end_date:
    dataset_date_str = str(dataset_date)
    dataset_date += datetime.timedelta(days=1)
    # choose a dataset type: clean dataset or original dataset
    # dataset_type = "-dataset.tsv.gz"
    dataset_type = "_clean-dataset.tsv.gz"
    dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/" + dataset_date_str + "/" + dataset_date_str + dataset_type + "?raw=true"

    if exists("clean-dataset.tsv.gz"):
        os.unlink("clean-dataset.tsv.gz")

    #Downloads the dataset (compressed in a GZ format)
    wget.download(dataset_URL, out='clean-dataset.tsv.gz')
    #Unzips the dataset and gets the TSV dataset
    with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
        with open('clean-dataset.tsv', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    #Deletes the compressed GZ file
    os.unlink("clean-dataset.tsv.gz")

    #Gets all possible languages from the dataset
    df = pd.read_csv('clean-dataset.tsv',sep="\t")
    lang_list = df.lang.unique()
    lang_list= sorted(np.append(lang_list,'all'))
    # pick language English
    lang_picker = widgets.Dropdown(options=lang_list, value="en")

    #Creates a new clean dataset with the specified language (if specified)
    filtered_language = lang_picker.value

    #If language is specified, it will create another tsv file with the filtered records
    filtered_tw = list()
    current_line = 1
    with open("clean-dataset.tsv") as tsvfile:
        tsvreader = csv.reader(tsvfile, delimiter="\t")

        if current_line == 1:
            filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))

            for line in tsvreader:
                if line[3] == filtered_language:
                    filtered_tw.append(linecache.getline("clean-dataset.tsv", current_line))
                current_line += 1

    if exists("clean-dataset-filtered.tsv.gz"):
        os.unlink("clean-dataset-filtered.tsv.gz")

    with open('clean-dataset-filtered.tsv', 'w') as f_output:
        for item in filtered_tw:
            f_output.write(item)
    linecache.clearcache()

    !python get_metadata.py -i clean-dataset-filtered.tsv -o $dataset_date_str -k api_keys.json

    # hydrated_tweets is a text file storing hydrated tweets (including unnecessary information)
  
    linecache.clearcache()

In [ ]:
# The number of sample to be selected from the dataset
num_samples = "1000"
list_tweets = None

# Change the date of dataset here
selected_date = datetime.date(2021,1,1)
selected_date_str = selected_date + '_short.json'

with open(selected_date_str, "r") as myfile:
    list_tweets = list(myfile)

if int(num_samples) > len(list_tweets):
    num_samples = len(list_tweets)

sample = random.sample(list_tweets, int(num_samples))

selected_date_sample = selected_date + '_sample_data.json'
file = open("selected_date_sample", "w")
for i in sample:
  file.write(i)
file.close() #This close() is important


In [ ]:
# Parse json
!python parse_json_lite.py sample_data.json p